# 모델링 전 사전세팅

In [44]:
# EDA 사전 세팅
%matplotlib inline

import os
os.environ['KMP_WARNINGS'] = 'off'
from functools import reduce
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
sns.set(font="AppleGothic", 

        rc={"axes.unicode_minus":False},
        
        style='darkgrid')

# 모델링
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import mean_absolute_error


from xgboost import plot_importance

In [4]:
!pip install scikit-optimize

# 데이터 사전준비

In [3]:
train = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_train_df.csv")
train.head()
valid = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_valid_df.csv")
train.head()

,car,reality,income_type,family_type,house_type,occyp_type,child_num,month_ap,family-child,family_size,income_total,DAYS_EMPLOYED,begin_month,new_age,aver_dep,year_income,y_train
0,0.0,1.0,0.0,0.0,1.0,8.0,0.648309,-0.730153,0.523754,0.764967,0.508421,-0.459964,-1.277577,-0.983449,-0.262049,0.155101,1
1,1.0,1.0,4.0,1.0,1.0,10.0,-0.661385,-0.730153,0.523754,-0.310591,2.476441,0.775603,-0.245901,1.210436,1.622181,-0.222538,2
2,0.0,1.0,0.0,1.0,1.0,14.0,-0.661385,1.026394,0.523754,-0.310591,0.071083,-0.224292,0.664401,0.061258,-0.014124,-0.176060,0
3,1.0,1.0,2.0,1.0,1.0,10.0,-0.661385,1.319152,0.523754,-0.310591,-0.366255,-0.218742,-0.003154,0.061258,-0.311634,-0.315496,2
4,0.0,1.0,4.0,1.0,1.0,6.0,1.958003,1.319152,0.523754,1.840524,0.727090,1.015544,-0.488648,-0.356625,-0.460389,-0.481750,1


In [44]:
#X_data = train.iloc[:, :-1]
#y_label = train.iloc[:, -1]
#X_train , X_valid , y_train , y_valid = train_test_split(X_data , y_label , test_size=0.2)

In [4]:
X_train = train.iloc[:, :-1]
X_valid = valid.iloc[:, :-1]
y_train = train.iloc[:, -1]
y_valid = valid.iloc[:, -1]

# 모델 간략히 돌려보기 <bn> Hyperparameter optimization X

In [43]:
def model_all(X_train, X_valid, y_train, y_valid):
    print('----------------------------------------스태킹 모델----------------------------------------')
    # 개별 ML 모델을 위한 Classifier 생성.
    knn_clf  = KNeighborsClassifier(n_neighbors=4)
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
    dt_clf = DecisionTreeClassifier()
    ada_clf = AdaBoostClassifier(n_estimators=100)

    # 최종 Stacking 모델을 위한 Classifier생성. 
    lr_final = LogisticRegression(C=10)
    
    # 개별 모델들을 학습. 
    knn_clf.fit(X_train, y_train); knn_pred = knn_clf.predict(X_valid)
    rf_clf.fit(X_train , y_train); rf_pred = rf_clf.predict(X_valid)
    dt_clf.fit(X_train , y_train); dt_pred = dt_clf.predict(X_valid)
    ada_clf.fit(X_train, y_train); ada_pred = ada_clf.predict(X_valid)
    
    
    print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_valid, knn_pred)))
    print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_valid, rf_pred)))
    print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_valid, dt_pred)))
    print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(y_valid, ada_pred)))
    
    pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
    # transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦. 
    pred = np.transpose(pred)
    
    lr_final.fit(pred, y_valid)
    final = lr_final.predict(pred)

    print('스태킹 최종 메타 모델의 예측 정확도: {0:.4f}%'.format(round(accuracy_score(y_valid , final) *100, 2)))
    
    print('\n');
    print('----------------------------------Logistic Regression-----------------------------------')
    
    lr = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=0, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
    
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_valid)
    acc = accuracy_score(y_valid, y_pred)
    print("Logistic Regression Accuracy: {:.2f}%".format(acc * 100))
    print('\n');
    print('-----------------------------------SGD (SGDClassifier)----------------------------------')
    
    sgd = SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
    
    sgd.fit(X_train, y_train);prediction = sgd.predict(X_valid)
    acc = accuracy_score(y_valid, prediction)
    print("SGDClassifier Accuracy: {:.2f}%".format(acc * 100))
    print('\n');
    print('-------------------------------KNN (KNeighborsClassifier)-------------------------------')
    
    knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')
    
    knn.fit(X_train, y_train);prediction = knn.predict(X_valid)
    acc = accuracy_score(y_valid, prediction)
    print("KNeighborsClassifier Accuracy: {:.2f}%".format(acc * 100))
    print('\n');
    print('-------------------------Decision Tree (DecisionTreeClassifier)-------------------------')
    
    dt = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       random_state=0, splitter='best')
    
    dt.fit(X_train, y_train);dt_pred = dt.predict(X_valid)
    
    acc = accuracy_score(y_valid, dt_pred)
    print("DecisionTreeClassifier Accuracy: {:.2f}%".format(acc * 100))
    
    print('\n');
    print('-----------------------------Cat Boost (Cat BoostClassifier)----------------------------')
    cat_model = CatBoostClassifier(iterations=100, logging_level='Silent')

    # 분류기를 훈련합니다.
    cat_model.fit(X_train, y_train)

    # 테스트 데이터에서 분류기를 사용하여 예측합니다.
    y_pred = cat_model.predict(X_valid)

    # 정확도를 계산합니다.
    acc = accuracy_score(y_valid, y_pred)
    print("CatBoostClassifier Accuracy: {:.2f}%".format(acc * 100))
    
    print('\n');
    print('---------------------------------XGBoost (XGBClassifier)--------------------------------')
    # create the model
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    print("XGBClassifier Accuracy: %.2f%%" % (accuracy * 100.0))
    
    print('\n');
    print('--------------------------LGBM(Light Gradient Boosting Machine)-------------------------')
    model = LGBMClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, y_pred)
    print("LGBMClassifier Accuracy: %.2f%%" % (accuracy * 100.0))

model_all(X_train, X_valid, y_train, y_valid)

----------------------------------------스태킹 모델----------------------------------------
KNN 정확도: 0.4850
랜덤 포레스트 정확도: 0.6857
결정 트리 정확도: 0.5976
에이다부스트 정확도: 0.6026 :
스태킹 최종 메타 모델의 예측 정확도: 63.6200%


----------------------------------Logistic Regression-----------------------------------
Logistic Regression Accuracy: 45.30%


-----------------------------------SGD (SGDClassifier)----------------------------------
SGDClassifier Accuracy: 52.87%


-------------------------------KNN (KNeighborsClassifier)-------------------------------
KNeighborsClassifier Accuracy: 46.50%


-------------------------Decision Tree (DecisionTreeClassifier)-------------------------
DecisionTreeClassifier Accuracy: 59.76%


-----------------------------Cat Boost (Cat BoostClassifier)----------------------------
CatBoostClassifier Accuracy: 66.68%


---------------------------------XGBoost (XGBClassifier)--------------------------------
[01:16:41] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948

## Catboost 하이퍼 파라미터 튜닝 GridSearchCV

In [19]:
param_grid = {'iterations': [100, 200, 300],
              'depth': [3, 4, 5],
              'learning_rate': [0.1, 0.05, 0.01],
              'l2_leaf_reg': [3, 1, 5]}

In [20]:
# CatBoost 분류기를 생성합니다.
cat_model = CatBoostClassifier()

In [21]:
grid_search = GridSearchCV(cat_model, param_grid, cv=5, verbose = 0)
grid_search.fit(X_train, y_train, verbose=0)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7fab7321a730>,
             param_grid={'depth': [3, 4, 5], 'iterations': [100, 200, 300],
                         'l2_leaf_reg': [3, 1, 5],
                         'learning_rate': [0.1, 0.05, 0.01]})

In [22]:
print("Best parameters: {}".format(grid_search.best_params_))

Best parameters: {'depth': 5, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


In [23]:
best_model = grid_search.best_estimator_

In [24]:
best_model
y_pred = best_model.predict(X_valid)

acc = accuracy_score(y_valid, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 68.96%


## Catboost 하이퍼 파라미터 튜닝 BayesSearchCV

In [25]:
from catboost import CatBoostClassifier
from skopt import BayesSearchCV

In [26]:
# 하이퍼파라미터 검색 범위를 설정합니다.
param_dist = {'iterations': (100, 300, 500),
              'depth': (2, 5, 10),
              'learning_rate': (0.01, 0.05, 0.1),
              'l2_leaf_reg': (1, 3, 5)}

# CatBoost 분류기를 생성합니다.
cat_model = CatBoostClassifier()

In [27]:
# Bayesian Optimization을 사용하여 하이퍼파라미터를 튜닝합니다.
bayes_search = BayesSearchCV(cat_model, param_dist, n_iter=50, verbose=0)
bayes_search.fit(X_train, y_train, verbose=0)

/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated

BayesSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7fab69657a90>,
              search_spaces={'depth': (2, 5, 10), 'iterations': (100, 300, 500),
                             'l2_leaf_reg': (1, 3, 5),
                             'learning_rate': (0.01, 0.05, 0.1)})

In [28]:
# 가장 좋은 하이퍼파라미터를 출력합니다.
print("Best parameters: {}".format(bayes_search.best_params_))

Best parameters: OrderedDict([('depth', 10), ('iterations', 500), ('l2_leaf_reg', 1), ('learning_rate', 0.1)])


In [29]:
# 가장 좋은 성능을 내는 모델을 얻습니다.
best_model = bayes_search.best_estimator_

In [30]:
y_pred = best_model.predict(X_valid)

acc = accuracy_score(y_valid, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 68.46%


## XGBClassifier RandomizedSearchCV

In [13]:
from sklearn.model_selection import RandomizedSearchCV

# define the parameter grid
param_grid = {
    'learning_rate': np.linspace(0.01, 0.3, num=30),
    'max_depth': [3, 4, 5, 6, 7, 8, 9],
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'subsample': np.linspace(0.5, 0.9, num=5),
}

# create the model
model = XGBClassifier()

In [16]:
# create the randomized search cross-validator
rs_cv = RandomizedSearchCV(model, param_grid, cv=5, n_iter=50, verbose=0)

# fit the randomized search cross-validator on the data
rs_cv.fit(X_train, y_train, verbose=0)

[03:09:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:09:23] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:09:41] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:09:58] WARNING: /Users/runner/miniforge3/c

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints...
                                           validate_parameters=None,
                                           verbosity=None),
                   n_iter=50,
                   param_distributions={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 ]),
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9],
                                        'n_estimators': [50, 100, 200, 300, 400,
                                                         500],
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])})

In [17]:
# print the best hyperparameters
print(rs_cv.best_params_)

# make predictions on the test set
y_pred = rs_cv.predict(X_valid)

# evaluate predictions
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

{'subsample': 0.6, 'n_estimators': 400, 'max_depth': 9, 'learning_rate': 0.24999999999999997}
Accuracy: 68.41%


## LGBM(Light Gradient Boosting Machine) Hyperparameter optimization with GridSearchCV

In [12]:
# define the parameter grid
param_grid = {
    'learning_rate': [0.1, 0.05],
    'max_depth': [3, 4, 5],
    'n_estimators': [200, 300],
    'subsample': [0.8, 1.0],
}

# create the model
model = LGBMClassifier()

In [13]:
# create the grid search cross-validator
gs_cv = GridSearchCV(model, param_grid, cv=5, verbose=0)

# fit the grid search cross-validator on the data
gs_cv.fit(X_train, y_train, verbose=0)

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.1, 0.05], 'max_depth': [3, 4, 5],
                         'n_estimators': [200, 300], 'subsample': [0.8, 1.0]})

In [14]:
# print the best hyperparameters
print(gs_cv.best_params_)

# make predictions on the test set
y_pred = gs_cv.predict(X_valid)

# evaluate predictions
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
Accuracy: 70.05%
